In [1]:
print('started')

started


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
np.__version__

'1.26.0'

In [4]:
pd.__version__

'2.1.1'

In [5]:
os.getcwd()

'/Users/yavuuzsameet/Desktop/polimi/tesi/recombinhunt-cov-7.0.0/fromzero'

In [6]:
# load the metadata
df = pd.read_csv('metadata_cleaned.tsv', sep='\t')
df.head()

/var/folders/2n/gzd0gjnd7zg2s5l8kbwzsr7h0000gn/T/ipykernel_3270/2889397875.py:2: DtypeWarning: Columns (4,10,29,52,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('metadata_cleaned.tsv', sep='\t')


,strain,virus,gisaid_epi_isl,genbank_accession,genbank_accession_rev,sra_accession,date,region,country,division,...,QC_rare_mutations,QC_snp_clusters,QC_frame_shifts,QC_stop_codons,QC_overall_score,QC_overall_status,frame_shifts,aaSubstitutions,clock_deviation,mutations
0,ARG/Cordoba-1006-155/2020,ncov,?,MW553298,MW553298.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,good,good,0.000000,good,NaN,"ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
1,ARG/Cordoba-1083-6/2020,ncov,?,MW553296,MW553296.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,good,good,0.173611,good,NaN,"ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",5.0,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
2,ARG/Cordoba-11301-61/2020,ncov,?,MW553314,MW553314.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,good,good,0.000000,good,NaN,"ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
3,ARG/Cordoba-11419-61/2020,ncov,?,MW553297,MW553297.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,good,good,0.000000,good,NaN,"ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
4,ARG/Cordoba-11499-61/2020,ncov,?,MW553316,MW553316.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,good,good,0.173611,good,NaN,"ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."


In [7]:
# show the example format of the data that should be created using the metadata above
example_path = '../demo/demo_input_nextstrain/samples_XAB.csv'
example = pd.read_csv(example_path, sep='\t' ,nrows=5)
example.head()

,name,true_lineage,nuc_changes
0,Denmark/DCGC-313405/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."
1,Denmark/DCGC-325735/2022,XAB,"11288_11296,21633_21641,28362_28370,241_C|T,28..."
2,Denmark/DCGC-405189/2022,XAB,"11288_11296,21633_21641,28362_28370,241_C|T,28..."
3,Denmark/DCGC-409430/2022,XAB,"6513_6515,11288_11296,28362_28370,241_C|T,2470..."
4,Denmark/DCGC-410519/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."


In [ ]:
# ONLY RUN AT THE VERY END
# # compare the result with the example
# result_path = 'reproduce/demo_input_nextstrain/samples_XAB.csv'
# result = pd.read_csv(result_path, sep='\t', nrows=5)
# result.head()

,name,true_lineage,nuc_changes
0,Denmark/DCGC-409430/2022,XAB,"6513_6515,11288_11296,28362_28370,241_C|T,2470..."
1,Denmark/DCGC-415376/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."
2,Denmark/DCGC-496325/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."
3,Denmark/DCGC-497413/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."
4,Denmark/DCGC-514342/2022,XAB,"6513_6515,11288_11296,21633_21641,28362_28370,..."


In [8]:
# set the needed columns as selected features
features = ['strain', 'pango_lineage', 'mutations']

In [9]:
def is_recombinant(lineage_name):
    lineage_name = lineage_name.upper()
    return lineage_name.startswith('X') and not '.' in lineage_name

In [10]:
recombinant_rows = df[df['pango_lineage'].apply(is_recombinant)]
recombinant_rows = recombinant_rows[features]

In [11]:
recombinant_rows.head()

,strain,pango_lineage,mutations
11489,BHR/23690002470_S59_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
11501,BHR/24020002741_S53_L001/2024,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."
11711,BHR/390105511_S54_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,1912_C|T,2790..."
11743,BHR/490002072_S51_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
11748,BHR/50202278_S4_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."


In [12]:
recombinant_rows.shape

(5523, 3)

In [13]:
recombinant_rows.reset_index(drop=True, inplace=True)
recombinant_rows.head()

,strain,pango_lineage,mutations
0,BHR/23690002470_S59_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
1,BHR/24020002741_S53_L001/2024,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."
2,BHR/390105511_S54_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,1912_C|T,2790..."
3,BHR/490002072_S51_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
4,BHR/50202278_S4_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."


In [14]:
recombinant_rows = recombinant_rows.rename(columns={
    'strain': 'name',
    'pango_lineage': 'true_lineage',
    'mutations': 'nuc_changes'
})
recombinant_rows.head()

,name,true_lineage,nuc_changes
0,BHR/23690002470_S59_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
1,BHR/24020002741_S53_L001/2024,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."
2,BHR/390105511_S54_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,1912_C|T,2790..."
3,BHR/490002072_S51_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,1269_A|G,2790_C|T,3037..."
4,BHR/50202278_S4_L001/2023,XDA,"241_C|T,405_A|G,670_T|G,981_C|T,1269_A|G,2790_..."


In [15]:
from tqdm.auto import tqdm
from collections import defaultdict
import json

In [19]:
lineage_groups = list(recombinant_rows.groupby('true_lineage'))
total_groups = len(lineage_groups)

In [20]:
lineage_counts = defaultdict(int)

In [21]:
with tqdm(lineage_groups, desc='Processing Lineages', unit='file') as pbar:
    for lineage, group in pbar:
        output_path = f'./demo_input_nextstrain/samples_{lineage}.csv'
        group.to_csv(
            output_path,
            sep='\t',
            index=False,
            header=True
        )

        lineage_counts[lineage] = len(group)

        pbar.set_postfix({
            'Current': lineage,
            'Saved': f"{pbar.n + 1}/{total_groups}"
        })

with open('./demo_input_nextstrain/samples_total.json', 'w') as f:
    json.dump(lineage_counts, f)

print("\n✅ All recombinant samples saved!")

Processing Lineages:   0%|          | 0/98 [00:00<?, ?file/s]


✅ All recombinant samples saved!
